In [ ]:
# @title SUMA FIJA NR
# Script de Colab para cálculo de asignación fija no remunerativa ("Suma Fija NR")
# Modificado: Solo incluye casos con valor menor a 60,000 (proporcionales)
# Columnas actualizadas: Convenio, Sueldo Basico, Legajo_ID

## Configuración inicial

# Importación de librerías necesarias
import pandas as pd
import numpy as np
import logging
from google.colab import files
from typing import Tuple, Dict, List
import io
import time
import re

# ==================== CONFIGURACIÓN DE LOGGING ====================

# 1. Obtenemos el logger raíz
logger = logging.getLogger()
logger.setLevel(logging.DEBUG) # Establecemos el nivel mínimo de logs a capturar
# 2. Limpiamos handlers preexistentes que Colab pueda haber añadido
if logger.hasHandlers():
    logger.handlers.clear()
# 3. Creamos un formateador para unificar el estilo de los logs
formatter = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s')
# 4. Creamos el handler para escribir en el archivo 'debug_calculos.txt'
file_handler = logging.FileHandler('debug_calculos.txt', mode='w', encoding='utf-8')
file_handler.setLevel(logging.DEBUG) # Captura todos los niveles de log en el archivo
file_handler.setFormatter(formatter)
# 5. Añadimos ÚNICAMENTE el handler del archivo al logger
logger.addHandler(file_handler)

# ============================================================================

# Al inicio del script, después de importar
logger.info("✅ Script cargado y logger configurado correctamente")

# Constantes del proyecto
IMPORTE_FIJO = 60000  # Valor para personal full time

LEGAJOS_FIJOS = {
    15645, 15077, 15746, 15763, 15772,
    15665, 15684, 15716, 15723, 15729,
    15736, 15738
}

## Funciones de carga de datos

def cargar_archivo_excel() -> pd.DataFrame:
    """
    Solicita al usuario cargar un archivo Excel y devuelve un DataFrame

    Returns:
        pd.DataFrame: DataFrame con los datos del archivo Excel
    """
    try:
        uploaded = files.upload()
        file_name = list(uploaded.keys())[0]
        logger.info(f"Archivo '{file_name}' cargado exitosamente")

        # Leer el archivo Excel
        df = pd.read_excel(io.BytesIO(uploaded[file_name]))
        logger.info(f"DataFrame cargado con {len(df)} registros y {len(df.columns)} columnas")

        # Verificar que las columnas necesarias existan (nombres actualizados)
        columnas_requeridas = ['Sector', 'Puesto', 'Contrato', 'HsSemanales',
                               'Legajo_ID', 'Convenio', 'Sueldo Basico']

        for col in columnas_requeridas:
            if col not in df.columns:
                logger.error(f"Columna requerida '{col}' no encontrada en el archivo")
                raise ValueError(f"Columna requerida '{col}' no encontrada")

        logger.info("Todas las columnas requeridas están presentes")
        return df

    except Exception as e:
        logger.error(f"Error al cargar el archivo: {str(e)}")
        raise

def validar_datos(df: pd.DataFrame) -> bool:
    """Valida que los datos tengan el formato y tipos esperados"""
    try:
        # Validar que HsSemanales sea numérico
        if not pd.api.types.is_numeric_dtype(df['HsSemanales']):
            logger.error("La columna HsSemanales debe ser numérica")
            return False

        # Validar que Sueldo Basico sea numérico
        if not pd.api.types.is_numeric_dtype(df['Sueldo Basico']):
            logger.error("La columna Sueldo Basico debe ser numérica")
            return False

        # Validar que no haya valores negativos en horas o sueldos
        if (df['HsSemanales'] < 0).any() or (df['Sueldo Basico'] < 0).any():
            logger.error("No se permiten valores negativos en HsSemanales o Sueldo Basico")
            return False

        return True
    except Exception as e:
        logger.error(f"Error en validación de datos: {str(e)}")
        return False

def normalizar_texto(texto: str) -> str:
    """
    Normaliza los campos de texto del input de nómina 
    para asegurar comparaciones uniformes en las reglas de cálculo.

    “Todas las reglas internas están escritas en formato ya normalizado (minúsculas, sin tildes, sin caracteres especiales),
     para garantizar matching exacto contra el input transformado con normalizar_texto”.

    Operaciones realizadas:
    - Convierte a string (por seguridad).
    - Elimina espacios al inicio y al final.
    - Convierte todo a minúsculas.
    - Reemplaza vocales acentuadas y diéresis por su equivalente simple.
    - Reemplaza la 'ñ' por 'n' para evitar inconsistencias.
    - Remueve caracteres especiales redundantes (/, -, múltiples espacios).
    
    Args:
        texto (str): Valor original tomado de columnas como "Convenio", "Puesto" o "Sector".

    Returns:
        str: Texto normalizado, apto para comparación en reglas.

    Ejemplos:
        >>> normalizar_texto("Técnico de Laboratorio")
        "tecnico de laboratorio"
        
        >>> normalizar_texto("Recepcionista Cajero/a ")
        "recepcionista cajeroa"
        
        >>> normalizar_texto("  SANIDAD 108/75  ")
        "sanidad 108 75"
    """
    if pd.isna(texto):
        return ""

    # Convertir a string, bajar a minúsculas y quitar espacios extremos
    texto = str(texto).strip().lower()

    # Reemplazar tildes y diéresis
    reemplazos = {
        'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
        'ü': 'u', 'ñ': 'n'
    }
    for viejo, nuevo in reemplazos.items():
        texto = texto.replace(viejo, nuevo)

    # Eliminar caracteres especiales comunes y normalizar espacios
    texto = re.sub(r"[^a-z0-9\s]", " ", texto)  # deja letras, números y espacios
    texto = re.sub(r"\s+", " ", texto)  # colapsa múltiples espacios en uno

    return texto.strip()

def normalizar_texto(texto):
    """
    Normaliza texto para comparación: minúsculas, sin tildes, sin espacios extras
    Normaliza los campos de texto del input de nómina 
    para asegurar comparaciones uniformes en las reglas de cálculo.

    “Todas las reglas internas están escritas en formato ya normalizado (minúsculas, sin tildes, sin caracteres especiales),
     para garantizar matching exacto contra el input transformado con normalizar_texto”.

    Operaciones realizadas:
    - Convierte a string (por seguridad).
    - Convierte todo a minúsculas.
    - Reemplaza vocales acentuadas y diéresis por su equivalente simple.
    - Reemplaza la 'ñ' por 'n' para evitar inconsistencias.
    - Remueve caracteres especiales redundantes (/, -, múltiples espacios).
    
    Args:
        texto (str): Valor original tomado de columnas como "Convenio", "Puesto" o "Sector".

    Returns:
        str: Texto normalizado, apto para comparación en reglas.

    Ejemplos:
        >>> normalizar_texto("Técnico de Laboratorio")
        "tecnico de laboratorio"
        
        >>> normalizar_texto("Recepcionista Cajero/a ")
        "recepcionista cajeroa"
        """
    if pd.isna(texto):
        return ""

    # Convertir a string, minúsculas, quitar espacios
    texto = str(texto).strip().lower()

    # Remover tildes (opcional, para mayor robustez)
    reemplazos = {
        'á': 'a', 'é': 'e', 'í': 'i', 'ó': 'o', 'ú': 'u',
        'ü': 'u', 'ñ': 'n'
    }
    for viejo, nuevo in reemplazos.items():
        texto = texto.replace(viejo, nuevo)

    return texto

## Funciones de cálculo

def calcular_jornada_parcial(fila: pd.Series) -> float:
    """
    Calcula el valor para empleados de jornada parcial según convenio Sanidad 108/75
    """
    try:
        # Validar que HsSemanales sea numérico
        if not isinstance(fila['HsSemanales'], (int, float, np.number)):
            logger.warning(f"Legajo_ID {fila['Legajo_ID']}: HsSemanales no es numérico ({type(fila['HsSemanales'])})")
            return np.nan

        # Normalizar el nombre del puesto robustamente
        puesto_normalizado = normalizar_texto(fila['Puesto'])
        logger.debug(f"Legajo_ID {fila['Legajo_ID']}: Puesto='{fila['Puesto']}' → Normalizado='{puesto_normalizado}', HsSemanales={fila['HsSemanales']}")

        # Subregla a: Puestos administrativos y operativos (todo normalizado)
        puestos_a = {
            "administrativo/a control de ordenes",
            "asistente de emergencias",
            "asistente tecnico",
            "operario de logistica",  # ← ¡Sin acento!
            "recepcionista",
            "recepcionista cajero/a",
            "telefonista"
        }

        logger.debug(f"Legajo_ID {fila['Legajo_ID']}: ¿Está en puestos_a? {puesto_normalizado in puestos_a}")

        # Definir los puestos que NO deben tener 35 horas
        puestos_excluir_35_horas = {
            "telefonista",
            "recepcionista cajero/a",
            "operario de logistica", 
            "recepcionista",          
            "asistente tecnico"       
        }

        # Verificar si aplica la regla
        aplica_regla = (
            puesto_normalizado in puestos_a and
            fila['HsSemanales'] < 36 and
            not (puesto_normalizado in puestos_excluir_35_horas and fila['HsSemanales'] == 35)
        )

        if aplica_regla:
            valor = IMPORTE_FIJO / 48 * fila['HsSemanales']
            valor = round(valor, 2)  # Redondeo a 2 decimales
            logger.info(f"Legajo_ID {fila['Legajo_ID']}: Aplica regla 1a - Valor: {valor:.2f}")
            return valor

        # Subregla b: Puestos técnicos de laboratorio
        puestos_b = {
            "auxiliar tecnico",
            "bioquimico",
            "tecnico de laboratorio",
            "tecnico extraccionista"
        }

        logger.debug(f"Legajo_ID {fila['Legajo_ID']}: ¿Está en puestos_b? {puesto_normalizado in puestos_b}")

        if puesto_normalizado in puestos_b and fila['HsSemanales'] < 27:
            valor = IMPORTE_FIJO / 36 * fila['HsSemanales']
            logger.info(f"Legajo_ID {fila['Legajo_ID']}: Aplica regla 1b - Valor: {valor:.2f}")
            return valor

        # Subregla c: Puestos técnicos especializados
        puestos_c = {"tecnico", "tecnico pivot"}

        logger.debug(f"Legajo_ID {fila['Legajo_ID']}: ¿Está en puestos_c? {puesto_normalizado in puestos_c}")
        logger.debug(f"Legajo_ID {fila['Legajo_ID']}: ¿Horas < 18? {fila['HsSemanales'] < 18}")

        if puesto_normalizado in puestos_c and fila['HsSemanales'] < 18:
            valor = IMPORTE_FIJO / 18 * fila['HsSemanales']
            logger.info(f"Legajo_ID {fila['Legajo_ID']}: Aplica regla 1c - Valor: {valor:.2f}")
            return valor

        logger.debug(f"Legajo_ID {fila['Legajo_ID']}: No aplica a ninguna subregla de jornada parcial")
        return np.nan

    except Exception as e:
        logger.error(f"Error calculando jornada parcial para Legajo_ID {fila['Legajo_ID']}: {str(e)}")
        return np.nan

def calcular_full_guardia(fila: pd.Series) -> float:
    """
    Calcula el valor para empleados de full guardia según convenio Sanidad 108/75
    """
    try:
        # Normalizar sector y puesto
        sector_normalizado = normalizar_texto(fila['Sector'])
        puesto_normalizado = normalizar_texto(fila['Puesto'])

        # Verificar condiciones base
        if not (sector_normalizado == "laboratorio" and fila['Sueldo Basico'] == 0):
            return np.nan

        # Subregla a: Puestos administrativos (normalizados)
        puestos_a = {"administrativo/a de piso", "recepcionista"}

        if puesto_normalizado in puestos_a and fila['HsSemanales'] < 36:
            valor = IMPORTE_FIJO / 48 * fila['HsSemanales']
            logger.info(f"Legajo_ID {fila['Legajo_ID']}: Aplica regla 2a - Valor: {valor:.2f}")
            return valor

        # Subregla b: Puestos técnicos (normalizados)
        puestos_b = {"auxiliar tecnico", "bioquimico", "tecnico de laboratorio", "tecnico extraccionista"}

        if puesto_normalizado in puestos_b and fila['HsSemanales'] < 27:
            valor = IMPORTE_FIJO / 36 * fila['HsSemanales']
            logger.info(f"Legajo_ID {fila['Legajo_ID']}: Aplica regla 2b - Valor: {valor:.2f}")
            return valor

        # Si no aplica a subreglas pero cumple condiciones base, NO se incluye en output
        logger.debug(f"Legajo_ID {fila['Legajo_ID']}: Aplica full guardia sin proporcional - NO INCLUIR EN OUTPUT")
        return np.nan

    except Exception as e:
        logger.error(f"Error calculando full guardia para Legajo_ID {fila['Legajo_ID']}: {str(e)}")
        return np.nan

def calcular_fuera_convenio(fila: pd.Series) -> float:
    """
    Calcula el valor para empleados fuera de convenio
    """
    try:
        # Normalizar todas las comparaciones de texto
        convenio_normalizado = normalizar_texto(fila['Convenio'])
        puesto_normalizado = normalizar_texto(fila['Puesto'])
        sector_normalizado = normalizar_texto(fila['Sector'])

        # Verificar condiciones base (todo normalizado)
        if not (convenio_normalizado == "fuera de convenio" and
                puesto_normalizado == "bioquimico" and
                sector_normalizado == "laboratorio" and
                fila['Sueldo Basico'] == 0):
            return np.nan

        # Calcular proporcional solo si aplica (horas menores)
        if fila['HsSemanales'] < 27:
            valor = IMPORTE_FIJO / 36 * fila['HsSemanales']
            logger.info(f"Legajo_ID {fila['Legajo_ID']}: Aplica regla 3 con proporcional - Valor: {valor:.2f}")
            return valor
        else:
            # Si tiene 27 o más horas, no se incluye en el output
            logger.debug(f"Legajo_ID {fila['Legajo_ID']}: Aplica regla 3 pero sin proporcional - NO INCLUIR EN OUTPUT")
            return np.nan

    except Exception as e:
        logger.error(f"Error calculando fuera de convenio para Legajo_ID {fila['Legajo_ID']}: {str(e)}")
        return np.nan

def determinar_codigo_variable(fila: pd.Series) -> int:
    """
    Determina el código variable según el convenio del empleado
    """
    convenio_normalizado = normalizar_texto(fila['Convenio'])

    if convenio_normalizado == "sanidad 108/75":
        return 2947
    else:
        return 7630

## Función principal de procesamiento

def procesar_nomina(df: pd.DataFrame) -> pd.DataFrame:
    """
    Procesa la nómina completa aplicando las reglas de cálculo
    y solo incluye en el output los casos con valor menor a 60,000,
    EXCLUYENDO legajos fijos con pago completo.

    Args:
        df (pd.DataFrame): DataFrame con los datos de entrada

    Returns:
        pd.DataFrame: DataFrame con los resultados del cálculo (solo casos proporcionales)
    """
    logger.info("Iniciando procesamiento de nómina")
    logger.info("NOTA: Solo se incluirán en el output los casos con valor menor a 60,000")
    logger.info("NOTA: Legajos fijos excluidos del cálculo")

    # Lista de legajos con pago fijo completo (sin cálculo)
    LEGAJOS_FIJOS = {
        15645, 15077, 15746, 15763, 15772,
        15665, 15684, 15716, 15723, 15729,
        15736, 15738
    }

    # Para trazar qué legajos fueron efectivamente excluidos
    legajos_excluidos = []

    # Listas para almacenar resultados
    sectores = []
    puestos = []
    contratos = []
    hs_semanales = []
    legajos_id = []
    codigos_variable = []
    valores = []

    for _, fila in df.iterrows():
        legajo = fila['Legajo_ID']

        # 🔹 Exclusión directa de legajos fijos
        if legajo in LEGAJOS_FIJOS:
            logger.info(f"Legajo_ID {legajo}: Excluido del cálculo (cobra $60.000 completo)")
            legajos_excluidos.append(legajo)
            continue

        logger.debug(f"Procesando Legajo_ID {legajo}")

        # Normalizar todos los campos de texto para comparación
        convenio_normalizado = normalizar_texto(fila['Convenio'])
        puesto_normalizado = normalizar_texto(fila['Puesto'])
        sector_normalizado = normalizar_texto(fila['Sector'])

        logger.debug(f"Legajo_ID {legajo}: Convenio='{fila['Convenio']}' → '{convenio_normalizado}'")
        logger.debug(f"Legajo_ID {legajo}: Puesto='{fila['Puesto']}' → '{puesto_normalizado}'")
        logger.debug(f"Legajo_ID {legajo}: Sector='{fila['Sector']}' → '{sector_normalizado}'")

        valor_final = np.nan

        # Aplicar reglas según convenio (usando texto normalizado)
        if convenio_normalizado == "sanidad 108/75":
            # Regla 1: Jornada Parcial
            valor_final = calcular_jornada_parcial(fila)

            # Regla 2: Full Guardia (solo si no aplicó jornada parcial)
            if pd.isna(valor_final):
                valor_final = calcular_full_guardia(fila)

        elif (convenio_normalizado == "fuera de convenio" and
              puesto_normalizado == "bioquimico" and
              sector_normalizado == "laboratorio" and
              fila['Sueldo Basico'] == 0):
            # Regla 3: Fuera de convenio
            valor_final = calcular_fuera_convenio(fila)
        else:
            # No aplica a ninguna regla
            logger.debug(f"Legajo_ID {legajo}: No aplica a ninguna regla - NO INCLUIR EN OUTPUT")

        # Solo agregar al resultado si tiene un valor válido y menor a 60,000
        if not pd.isna(valor_final) and valor_final < IMPORTE_FIJO:
            sectores.append(fila['Sector'])
            puestos.append(fila['Puesto'])
            contratos.append(fila['Contrato'])
            hs_semanales.append(fila['HsSemanales'])
            legajos_id.append(legajo)

            codigo = determinar_codigo_variable(fila)
            codigos_variable.append(codigo)

            valores.append(valor_final)
            logger.info(f"Legajo_ID {legajo}: INCLUIDO EN OUTPUT - Valor: {valor_final:.2f}")
        else:
            logger.debug(f"Legajo_ID {legajo}: NO INCLUIDO (valor completo o no aplica)")

    # Crear DataFrame de resultados solo con los casos que aplican
    resultados = pd.DataFrame({
        'Sector': sectores,
        'Puesto': puestos,
        'Contrato': contratos,
        'HsSemanales': hs_semanales,
        'Legajo_ID': legajos_id,
        'CodVariable': codigos_variable,
        'Valor': valores
    })

    # REDONDEO A 2 DECIMALES - SOLO AQUÍ SE APLICA
    resultados['Valor'] = resultados['Valor'].round(2)

    logger.info(f"Procesamiento completado. {len(resultados)} empleados con asignación proporcional")
    logger.info(f"Valores redondeados a 2 decimales para formato monetario")

    # 🔹 Log consolidado con los legajos fijos excluidos
    if legajos_excluidos:
        logger.info(f"Legajos excluidos del cálculo (fijo $60.000): {sorted(legajos_excluidos)}")

    return resultados

## Ejecución del script
# Ejecución principal
df_nomina = pd.DataFrame() # Definir fuera del try para que exista en el finally
df_resultados = pd.DataFrame() # Definir fuera del try

try:
    logger.info("=== INICIO DEL PROCESO ===")

    # Cargar datos
    df_nomina = cargar_archivo_excel()

     # ✅ VALIDAR DATOS - NUEVA LLAMADA
    if not validar_datos(df_nomina):
        logger.error("Los datos no pasaron la validación. Abortando proceso.")
        raise ValueError("Datos inválidos en el archivo de entrada")

    # Procesar nómina
    df_resultados = procesar_nomina(df_nomina)

    # Guardar resultados en Excel
    nombre_archivo_resultados = "resultados_asignacion_nr.xlsx"

    if not df_resultados.empty:
        df_resultados.to_excel(nombre_archivo_resultados, index=False)
        logger.info(f"Archivo de resultados '{nombre_archivo_resultados}' generado con {len(df_resultados)} registros")
        files.download(nombre_archivo_resultados)
    else:
        logger.info("No se encontraron registros que cumplan los criterios para el output.")
        # Se puede generar un archivo vacío si se desea
        # df_resultados.to_excel(nombre_archivo_resultados, index=False)
        # files.download(nombre_archivo_resultados)

except Exception as e:
    logger.critical(f"Error fatal en el proceso principal: {str(e)}", exc_info=True)
    print(f"\n❌ OCURRIÓ UN ERROR CRÍTICO. Revisa el log para más detalles.")

finally:
    # Cierra y vacía todos los handlers del logging de forma segura.
    logging.shutdown()

    # Pausa para asegurar que el sistema de archivos de Colab registre el archivo.
    time.sleep(1)

    print("\n" + "="*50)
    print("RESUMEN EJECUCIÓN".center(50))
    print("="*50)

    if not df_nomina.empty:
        print(f"📊 Empleados procesados: {len(df_nomina)}")
        print(f"✅ Empleados con asignación proporcional: {len(df_resultados)}")
        print(f"❌ Empleados sin asignación: {len(df_nomina) - len(df_resultados)}")

        if not df_resultados.empty:
            print(f"💰 Total asignado: ${df_resultados['Valor'].sum():.2f}")
            print(f"📈 Valor promedio: ${df_resultados['Valor'].mean():.2f}")
            print(f"🔽 Valor mínimo: ${df_resultados['Valor'].min():.2f}")
            print(f"🔼 Valor máximo: ${df_resultados['Valor'].max():.2f}")
    else:
        print("❌ No se pudo cargar el archivo de nómina")

    print("="*50)
    print("=== PROCESO FINALIZADO ===".center(50))
    print("="*50)

    # Descargar log de debug
    try:
        print(f"\nDescargando archivo de log: debug_calculos.txt...")
        files.download('debug_calculos.txt')
    except Exception as e:
        print(f"⚠️ No se pudo descargar el archivo de log: {str(e)}")